In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torchtext import data
import re 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
label = pd.read_csv('label.csv') # Labelled Data provided
label['id'] = label['id'].apply(lambda x: x.split('-')[0]).astype(int)

[nltk_data] Downloading package stopwords to /home/gwan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
label.loc[label.annotation == 'rejected','annotation'] = 0.
label.loc[label.annotation != 0,'annotation'] = 1


In [68]:
label

,id,begin,end,text,annotation
0,1838283,477,483,NICU nursing progress note\n\n\nPlease refer t...,0.0
1,1573658,98,106,"CCU Nursing Progress Note\nS-""I still feel ali...",1
2,380371,2977,2988,TITLE:\n Chief Complaint: Declining mental ...,0.0
3,1533008,494,500,Neuro: Pt movements more vigorous. Eyes open...,0.0
4,1472567,1178,1188,SICU Nsg Note\nNeuro - To CT this am for scan ...,1
...,...,...,...,...,...
527,441775,1771,1781,Chief Complaint: rigors\n 24 Hour Events:\n ...,1
528,1465223,453,459,7p to 7a Micu Progress Note\n\nNeuro - Pt rema...,0.0
529,683821,4784,4793,"Chief Complaint: food stuck in throat, fevers...",0.0
530,1795496,602,608,PCA note 0700-1900\n\n\n1. Resp: [** 360**] re...,0.0


In [69]:
from numpy import int64


label = label[['id','text','annotation']]
label['annotation'] = label['annotation'].astype(int64)

/tmp/ipykernel_21836/393104755.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label['annotation'] = label['annotation'].astype(int64)


In [70]:
def cleanup_text(texts):
    text = texts
    en_stops = set(stopwords.words('english'))
    
    # Lower case
    text = text.lower()
     # remove punctuation
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    # remove multiple spaces
    text = re.sub(r' +', ' ', text)
    # remove newline
    text = re.sub(r'\n', ' ', text)
    # Remove stopwords
    filtered_words = [word for word in text.split(' ') if word not in stopwords.words('english')]
    return ' '.join(filtered_words)

In [71]:
label['text'] = label['text'].apply(cleanup_text)

# Split data
validation_percent = 0.20

# Split data into train / validate
train,test = train_test_split(label,test_size=validation_percent,random_state = 10)

/tmp/ipykernel_21836/4204269916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label['text'] = label['text'].apply(cleanup_text)


In [73]:
label.text

0      nicu nursing progress note please refer flowsh...
1      ccu nursing progress note still feel alittle n...
2      title chief complaint declining mental status ...
3      neuro pt movements vigorous eyes open often re...
4      sicu nsg note neuro ct scan head showed atroph...
                             ...                        
527    chief complaint rigors 24 hour events 3 12 eve...
528    7p 7a micu progress note neuro pt remains mini...
529    chief complaint food stuck throat fevers cough...
530    pca note 0700 1900 1 resp 360 remains ra rr 30...
531    full code nkda 55 f morbid obesity htn recent ...
Name: text, Length: 532, dtype: object

In [74]:
train.to_csv('train.csv',index = False)
test.to_csv('test.csv',index =False)